# Multisession registration with CaImAn

This notebook will help to demonstrate how to use CaImAn on movies recorded in multiple sessions. CaImAn has in-built functions that align movies from two or more sessions and try to recognize components that are imaged in some or all of these recordings.

The basic function for this is `caiman.base.rois.register_ROIs()`. It takes two sets of spatial components and finds components present in both using an intersection over union metric and the Hungarian algorithm for optimal matching.
`caiman.base.rois.register_multisession()` takes a list of spatial components, aligns sessions 1 and 2, keeps the union of the matched and unmatched components to register it with session 3 and so on.

In [167]:
from IPython import get_ipython
from matplotlib import pyplot as plt
import numpy as np
import h5py
import glob
import pims_nd2
import pandas as pd
from collections import OrderedDict
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns
import os

from caiman.base.rois import register_multisession
from caiman.utils import visualization
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.utils.utils import download_demo
import sys
sys.path.append('..')
from placecode.cross_registration_functions import CellTrackingSingleAnimal as celltrack

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


loading the files and trying to make everything automatic

In [168]:
animal='OPI2342'
conditions=['novel_belt_d1','novel_belt_d2','novel_belt_d3']

In [169]:
# Load multisession data (spatial components and mean intensity templates) (should be replaced by actual data)
home_folder=f'D:/sd_project_pbox/exported_data/{animal}/'
raw_data_folder=f'D:/sd_project_pbox/raw_data/{animal}'
output_folder=f'D:/sd_project_pbox/results/{animal}/'

files_list=[]
templates_list=[]
cnmf_list=[]
spatials=[]
dims=[]
# spatials=[cnmf_1.estimates.A,cnmf_2.estimates.A,cnmf_3.estimates.A]
# dims=cnmf_1.dims

for condition in conditions:

    #finding file paths
    fpath= glob.glob(f"{home_folder}/{animal}_{condition}/*.hdf5")[0]
    files_list.append(fpath)

    #sstoring cnfs
    cnmf_ind=cnmf.load_CNMF(fpath,'r')
    cnmf_list.append(cnmf_ind)
    #sotring spatial components
    spatial=cnmf_ind.estimates.A
    spatials.append(spatial)
    #storing dims
    dim=cnmf_ind.dims
    dims.append(dim)

    #template paths
    # template_path=glob.glob(f"{raw_data_folder}/*{condition}/*.nd2")[0]
    # nikon_movie=pims_nd2.ND2_Reader(template_path)
    # template=np.mean(nikon_movie[:600],axis=0)
    # templates_list.append(template)


In [170]:
from matplotlib.colors import LinearSegmentedColormap

In [171]:
# cmap = LinearSegmentedColormap.from_list('custom_cmap', ['black', 'red'], N=2)

# plt.imshow(template3,cmap=cmap)

## Use `register_multisession()`

The function `register_multisession()` requires 3 arguments:
- `A`: A list of ndarrays or scipy.sparse.csc matrices with (# pixels X # component ROIs) for each session
- `dims`: Dimensions of the FOV, needed to restore spatial components to a 2D image
- `templates`: List of ndarray matrices of size `dims`, template image of each session

In [172]:
spatial_union, assignments, matchings = register_multisession(A=spatials, dims=dims[0])#, templates=templates)

c:\Users\ttami\anaconda3\envs\caiman\lib\site-packages\scipy\sparse\_index.py:151: SparseEfficiencyWarning:

Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.



finding cells that were cross registered only for two sessions
dictionary is based on the first group that I am comparing two

In [173]:
cr_reg_cells_two_ses=OrderedDict()
df=pd.DataFrame(data=assignments,columns=conditions)
for i,condition in enumerate(conditions[:-1]):
    df_per_sessions=df[[conditions[i],conditions[i+1]]]
    df_per_sessions=df_per_sessions.dropna().astype(int)
    cr_reg_cells_two_ses[condition]=df_per_sessions
    

In [174]:
# Find the rows where there are no NaN values
#meaning where are the cells in all of the days
cr_reg_cells = assignments[~np.isnan(assignments).any(axis=1)].astype(int)
cr_reg_cells_df=pd.DataFrame(data=cr_reg_cells,columns=conditions)


In [175]:
cr_reg_cells_df

,novel_belt_d1,novel_belt_d2,novel_belt_d3
0,51,80,81
1,62,85,47
2,66,76,70
3,69,93,97
4,72,66,34
5,73,90,56
6,75,60,127
7,76,67,215


In [176]:
#find the number of cells that were place in all three conditions
output_file=f"D:/sd_project_pbox/results/{animal}"

#first store the place cells somewhere
pc_cells=OrderedDict()
non_pc_cells=OrderedDict()
silent_cells=OrderedDict()


for condition in conditions:
    #appending place cells
    pc_cells_per_day=h5py.File(f'{output_file}/{condition}/output_info.h5')['place_cells_tuned_vector']
    pc_cells_per_day=np.array(pc_cells_per_day)
    pc_cells[condition]=[]
    pc_cells[condition].append(pc_cells_per_day)

    #appending non place cells
    non_pc_cells_per_day=h5py.File(f'{output_file}/{condition}/output_info.h5')['non_place_cells_tuned_vector']
    non_pc_cells_per_day=np.array(non_pc_cells_per_day)
    non_pc_cells[condition]=[]
    non_pc_cells[condition].append(non_pc_cells_per_day)

    #appending silent cells
    silent_cells_per_day=h5py.File(f'{output_file}/{condition}/output_info.h5')['silent_cells']
    silent_cells_per_day=np.array(silent_cells_per_day)
    silent_cells[condition]=[]
    silent_cells[condition].append(silent_cells_per_day)


find cross registered lace cells between sessions 
eg baseline 1 to baseline 2
baseline 2 to 24h post etc

In [177]:
pc_cells_per_sessions=OrderedDict()

for i,condition in enumerate(conditions[:-1]):
    sessions=[condition,conditions[i+1]]
    df=cr_reg_cells_two_ses[condition]
    pc_cells_per_sessions[condition]=celltrack.filter_cross_registered_place_cells(df, sessions, pc_cells)
# Example usage for two sessions:
#filtered_df = celltrack.filter_cross_registered_place_cells(df, sessions, pc_cells)


store the avergafe firing rate maps only of the cross registered place cells between sessions

In [178]:
#indetify only the cross registered place cells in each of the recordings
fr_rate_maps_to_compare=OrderedDict()

for j,condition in enumerate(conditions[:-1]):
    fr_rate_maps_to_compare[f'{condition}_{conditions[j+1]}']=[]
    avr_fr_rate_map_init=h5py.File(f'{output_folder}/{condition}/output_info.h5')['avr_firing_rate_maps']
    avr_fr_rate_map_end=h5py.File(f'{output_folder}/{conditions[j+1]}/output_info.h5')['avr_firing_rate_maps']
    avr_fr_rate_map_init=pd.DataFrame(avr_fr_rate_map_init)
    avr_fr_rate_map_end=pd.DataFrame(avr_fr_rate_map_end)
    cr_reg_cells_values_init=pc_cells_per_sessions[condition].iloc[:,0].values
    avr_fr_rate_map_init=avr_fr_rate_map_init.iloc[cr_reg_cells_values_init]
    cr_reg_cells_values_end=pc_cells_per_sessions[condition].iloc[:,1].values
    avr_fr_rate_map_end=avr_fr_rate_map_end.iloc[cr_reg_cells_values_end]
    fr_rate_maps_to_compare[f'{condition}_{conditions[j+1]}'].extend([avr_fr_rate_map_init,avr_fr_rate_map_end])



In [180]:
#plotting the original place cell coding diagrams
n_sessions_to_compare=2

sel_fontize=20
for j,condition in enumerate(conditions[:-1]):
    sessions=f'{condition}_{conditions[j+1]}'

    #if there not place cells between the session its doesnt plot anything
    if fr_rate_maps_to_compare[sessions][0].empty:
        continue

    fig,ax=plt.subplots(2,n_sessions_to_compare,figsize=(20,20))
    fig.suptitle(f'{animal}\ncross registered place cells\n comparison\n{sessions}\n Up Original sorting\nBelow sorted to {condition}',fontsize=sel_fontize+5)
                        
    custom_ticks = [0, 50, 100, 150]  # Specify the positions where you want the ticks
    custom_labels = ['0', '50', '100', '150']  

    for i in range(n_sessions_to_compare):


        place_cell_activity=fr_rate_maps_to_compare[sessions][i]
        place_cell_indexes_max=np.argmax(place_cell_activity,axis=1) #finding the place of the max activity of every cell
        place_cell_indexed_filtered=place_cell_activity.apply(np.argmax, axis=1).sort_values().index #there are the new indexes of the place cells just so thez can form the nice place cell diagram
        transpose=place_cell_indexed_filtered
        place_cell_activity=place_cell_activity.reindex(index=transpose) #reindexing the place cell panda frame according to the max activitz and the position in the belt


        sns.heatmap(place_cell_activity,ax=ax[0,i],cmap='viridis',cbar=False)#,vmax=4)
        if i==0:
            ax[0,i].set_ylabel('cross registered place cells',fontsize=sel_fontize)
        else:
            ax[0,i].set_ylabel('')
        ax[0,i].set_xlabel('')
        ax[0,i].set_yticks([])
        ax[0,i].set_yticklabels([])
        ax[0,i].set_title(conditions[j+i],fontsize=sel_fontize)
        ax[0,i].set_xticks([])
        ax[0,i].set_xticklabels([])

      
    #plotting place coding diagram transposed on the first one

    place_cell_activity=fr_rate_maps_to_compare[sessions][0].reset_index(drop=True)
    place_cell_indexes_max=np.argmax(place_cell_activity,axis=1) #finding the place of the max activity of every cell
    place_cell_indexed_filtered=place_cell_activity.apply(np.argmax, axis=1).sort_values().index 
    transpose=place_cell_indexed_filtered

    for i in range(n_sessions_to_compare):
        place_cell_activity=fr_rate_maps_to_compare[sessions][i].reset_index(drop=True)
        place_cell_activity=place_cell_activity.reindex(index=transpose)
        sns.heatmap(place_cell_activity,ax=ax[1,i],cmap='viridis',cbar=False)
        if i==0:
            ax[1,i].set_ylabel('cross registered place cells',fontsize=sel_fontize)
        else:
            ax[1,i].set_ylabel('')
        ax[1,i].set_xlabel('Belt(cm)',fontsize=sel_fontize)
        ax[1,i].set_yticks([])
        ax[1,i].set_yticklabels([])
        ax[1,i].set_xticks(custom_ticks)
        ax[1,i].set_xticklabels(custom_labels,fontsize=sel_fontize)


    plt.tight_layout()

    save_folder=f'{output_folder}/cross_registration'

    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    plt.savefig(f'{save_folder}/{animal}_{sessions}_place_cells_comparison.png', format='png', dpi=300, bbox_inches='tight')
    plt.savefig(f'{save_folder}/{animal}_{sessions}_place_cells_comparison.pdf', format='pdf', dpi=300, bbox_inches='tight')

In [181]:
#plotting the original place cell coding diagrams

# sel_fontize=20
# for j,condition in enumerate(conditions[:-1]):
#     sessions=[conditions[j],conditions[j+1]]

#     fig,ax=plt.subplots(2,len(sessions),figsize=(20,20))
#     fig.suptitle(f'{animal}\ncross registered place cells\n comparison{sessions}\n Up Original sorting\nBelow sorted to {sessions[0]}',fontsize=sel_fontize+5)
                        
#     custom_ticks = [0, 50, 100, 150]  # Specify the positions where you want the ticks
#     custom_labels = ['0', '50', '100', '150']  

#     for i,session in enumerate(sessions):
#         place_cell_activity=fr_rate_maps_to_compare[session]
#         place_cell_indexes_max=np.argmax(place_cell_activity,axis=1) #finding the place of the max activity of every cell
#         place_cell_indexed_filtered=place_cell_activity.apply(np.argmax, axis=1).sort_values().index #there are the new indexes of the place cells just so thez can form the nice place cell diagram
#         transpose=place_cell_indexed_filtered
#         place_cell_activity=place_cell_activity.reindex(index=transpose) #reindexing the place cell panda frame according to the max activitz and the position in the belt


#         sns.heatmap(place_cell_activity,ax=ax[0,i],cmap='viridis',cbar=False)#,vmax=4)
#         if i==0:
#             ax[0,i].set_ylabel('cross registered place cells',fontsize=sel_fontize)
#         else:
#             ax[0,i].set_ylabel('')
#         ax[0,i].set_xlabel('')
#         ax[0,i].set_yticks([])
#         ax[0,i].set_yticklabels([])
#         ax[0,i].set_title(session,fontsize=sel_fontize)
#         ax[0,i].set_xticks([])
#         ax[0,i].set_xticklabels([])

#     #plotting place coding diagram transposed on the first one

#     place_cell_activity=fr_rate_maps_to_compare[sessions[0]].reset_index(drop=True)
#     place_cell_indexes_max=np.argmax(place_cell_activity,axis=1) #finding the place of the max activity of every cell
#     place_cell_indexed_filtered=place_cell_activity.apply(np.argmax, axis=1).sort_values().index 
#     transpose=place_cell_indexed_filtered

#     for i,session in enumerate(sessions):
#         place_cell_activity=fr_rate_maps_to_compare[session].reset_index(drop=True)
#         place_cell_activity=place_cell_activity.reindex(index=transpose) #reindexing the place cell panda frame according to the max activitz and the position in the belt

#         sns.heatmap(place_cell_activity,ax=ax[1,i],cmap='viridis',cbar=False)
#         if i==0:
#             ax[1,i].set_ylabel('cross registered place cells',fontsize=sel_fontize)
#         else:
#             ax[1,i].set_ylabel('')
#         ax[1,i].set_xlabel('Belt(cm)',fontsize=sel_fontize)
#         ax[1,i].set_yticks([])
#         ax[1,i].set_yticklabels([])
#         ax[1,i].set_xticks(custom_ticks)
#         ax[1,i].set_xticklabels(custom_labels,fontsize=sel_fontize)

#     plt.tight_layout()

#     save_folder=f'{output_folder}/cross_regisration'

#     if not os.path.exists(save_folder):
#         os.makedirs(save_folder)

#     plt.savefig(f'{save_folder}/{animal}_{sessions}_place_cells_comparison.png', format='png', dpi=300, bbox_inches='tight')
#     plt.savefig(f'{save_folder}/{animal}_{sessions}_place_cells_comparison.pdf', format='pdf', dpi=300, bbox_inches='tight')

In [183]:
labels = [] 
colors = []
for condition in conditions:
  labels.extend([f"PC {condition}", f"nPC {condition}", f"sil {condition}"])  # for each condition, check categories PC, not-PC and low activity
  colors.extend(["red", "blue", "grey"])  # 255, 0, 0;  0, 255, 0; 0, 0, 0
# in each condition, we have PC and nPC categories, each have PC and nPC targets in the next category
sources = []  # should be 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 4, 5, 3, 4, 5, 3, 4, 5, ...
targets = []  # should be 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, ...
values = []
link_colors = []
df=cr_reg_cells_df

for i_condition in range(len(conditions)-1): 

  #target cells place cells
  n_PC_to_PC=len(celltrack.cell_movement(df,conditions[i_condition],conditions[i_condition+1],source_type=pc_cells,target_type=pc_cells))
  n_non_PC_to_PC=len(celltrack.cell_movement(df,conditions[i_condition],conditions[i_condition+1],source_type=non_pc_cells,target_type=pc_cells))
  n_sil_to_PC=len(celltrack.cell_movement(df,conditions[i_condition],conditions[i_condition+1],source_type=silent_cells,target_type=pc_cells))
  sources.extend([3*i_condition,3*i_condition+1, 3*i_condition+2])
  targets.extend([3*(i_condition+1), 3*(i_condition+1), 3*(i_condition+1)])
  values.extend([n_PC_to_PC, n_non_PC_to_PC, n_sil_to_PC])
  link_colors.extend(["rgba(255, 0, 0, 0.4)", "rgba(0, 0, 255, 0.4)", "rgba(0, 0, 0, 0.4)"])  # PC -> x is light blue, nPC -> x is light red, Q -> y is "light black"

  #target cells non place cells
  n_PC_to_non_PC=len(celltrack.cell_movement(df,conditions[i_condition],conditions[i_condition+1],source_type=pc_cells,target_type=non_pc_cells))
  n_sil_to_non_PC=len(celltrack.cell_movement(df,conditions[i_condition],conditions[i_condition+1],source_type=silent_cells,target_type=non_pc_cells))
  n_non_PC_to_non_PC=len(celltrack.cell_movement(df,conditions[i_condition],conditions[i_condition+1],source_type=non_pc_cells,target_type=non_pc_cells))
  sources.extend([3*i_condition,3*i_condition+1, 3*i_condition+2])
  targets.extend([3*(i_condition+1)+1, 3*(i_condition+1)+1, 3*(i_condition+1)+1])
  values.extend([n_PC_to_non_PC, n_non_PC_to_non_PC, n_sil_to_non_PC])
  link_colors.extend(["rgba(255, 0, 0, 0.4)", "rgba(0, 0, 255, 0.4)", "rgba(0, 0, 0, 0.4)"])  # PC -> x is light blue, nPC -> x is light red, Q -> y is "light black"


  #targer cell silent cells  
  n_PC_to_sil=len(celltrack.cell_movement(df,conditions[i_condition],conditions[i_condition+1],source_type=pc_cells,target_type=silent_cells))
  n_non_PC_to_sil=len(celltrack.cell_movement(df,conditions[i_condition],conditions[i_condition+1],source_type=non_pc_cells,target_type=silent_cells))
  n_sil_to_sil=len(celltrack.cell_movement(df,conditions[i_condition],conditions[i_condition+1],source_type=silent_cells,target_type=silent_cells))

  sources.extend([3*i_condition,3*i_condition+1, 3*i_condition+2])
  targets.extend([3*(i_condition+1)+2, 3*(i_condition+1)+2, 3*(i_condition+1)+2])
  values.extend([n_PC_to_sil, n_non_PC_to_sil, n_sil_to_sil])
  link_colors.extend(["rgba(255, 0, 0, 0.4)", "rgba(0, 0, 255, 0.4)", "rgba(0, 0, 0, 0.4)"])  # PC -> x is light blue, nPC -> x is light red, Q -> y is "light black"

  
fig = go.Figure(data=[go.Sankey(
  node = dict(
    pad = 15,
    thickness = 20,
    line = dict(color = "black", width = 0.5),
    label = labels,
    color = colors
  ),
  link = dict(
    source = sources, # indices correspond to labels, eg A1, A2, A1, B1, ...
    target = targets,
    value = values,
    color = link_colors
))])

fig.update_layout(title_text=f'{animal}\n\n{conditions}', font_size=10)

save_folder=f'D:/sd_project_pbox/results/{animal}/cross_registration'

if not os.path.exists(save_folder):
  os.makedirs(save_folder)

fig.write_html(f'D:/sd_project_pbox/results/{animal}/cross_registration/{conditions}.html')


#pio.write_image(fig, F'{output_folder}/{animal}_sankey_diagram.pdf')

